In [131]:
import pandas as pd

# import techminer.jupyter as jpl
from techminer.analytics import load_scopus

filepath = "../data/papers/urban-agriculture.csv"
df = pd.read_csv(filepath)
df = load_scopus(df)

# df = load_scopus(df)
# df.info()
# df = df.applymap(lambda x: None if pd.isna(x) is True else x)
# df = DataFrame(df).generate_ID()

100%|██████████| 8/8 [00:12<00:00,  1.54s/it]


In [135]:
import ipywidgets as widgets
from ipywidgets import Layout

import techminer.analytics as tc
import techminer.plots as plt

# import matplotlib.pyplot as plt

In [136]:
COLORMAPS = [
    "Greys",
    "Purples",
    "Blues",
    "Greens",
    "Oranges",
    "Reds",
    "YlOrBr",
    "YlOrRd",
    "OrRd",
    "PuRd",
    "RdPu",
    "BuPu",
    "GnBu",
    "PuBu",
    "YlGnBu",
    "PuBuGn",
    "BuGn",
    "YlGn",
    "Pastel1",
    "Pastel2",
    "Paired",
    "Accent",
    "Dark2",
    "Set1",
    "Set2",
    "Set3",
    "tab10",
    "tab20",
    "tab20b",
    "tab20c",
]

In [137]:
def menu_descriptive_stats(x):
    return widgets.Text("TO-DO")

In [139]:
def menu_analysis_by_term_per_year(x):
    return widgets.Text("TO-DO")

In [140]:
def menu_analysis_by_term_per_term_per_year(x):
    return widgets.Text("TO-DO")

In [141]:
def menu_occurrence(x):
    return widgets.Text("TO-DO")

In [142]:
def menu_co_occurrence(x):
    return widgets.Text("TO-DO")

In [143]:
def menu_autocorrelation(x):
    return widgets.Text("TO-DO")

In [144]:
def menu_correlation(x):
    return widgets.Text("TO-DO")

In [145]:
def menu_factor_analysis(x):
    return widgets.Text("TO-DO")

In [168]:
def menu_analysis_by_year(x):
    def compute(selected, plot_type, cmap):
        df = tc.summary_by_year(x)
        df = df[data[selected]]
        plot = plots[plot_type]
        output.clear_output()
        with output:
            display(plot(df, cmap=cmap))
    #
    # Options
    #
    data = {
        "Documents by Year": ["Year", "Num Documents"],
        "Cum. Documents by Year": ["Year", "Num Documents (Cum)"],
        "Times Cited by Year": ["Year", "Times Cited"],
        "Cum. Times Cited by Year": ["Year", "Times Cited (Cum)"],
        "Avg. Times Cited by Year": ["Year", "Avg. Times Cited"],
    }
    plots = {'bar': plt.bar, "barh": plt.bar}
    #
    selected = widgets.Dropdown(
        options=list(data.keys()), value=list(data.keys())[0], description="Plot:", disable=False,
    )
    plot_type = widgets.Dropdown(
        options=["bar", "barh"], description="Plot type:", disable=False,
    )
    cmap = widgets.Dropdown(options=COLORMAPS, description="Colormap:", disable=False,)
    #
    output = widgets.Output()
    with output:
        display(
            widgets.interactive_output(
                compute, {"selected": selected, "plot_type": plot_type, "cmap": cmap}
            )
        )
    #
    left_box = widgets.VBox(
        [selected, plot_type, cmap], layout=Layout(height="350px", border="1px solid gray"),
    )
    right_box = widgets.VBox([output])
    return widgets.HBox([left_box, right_box])

In [138]:
def menu_analysis_by_term(x):
    def compute(selected, plot_type, cmap):
        df = tc.summary_by_year(x)
        df = df[data[selected]]
        plot = plots[plot_type]
        output.clear_output()
        with output:
            display(plot(df, cmap=cmap))
    #
    # Options
    #
    data = {
        "Documents by Year": ["Year", "Num Documents"],
        "Cum. Documents by Year": ["Year", "Num Documents (Cum)"],
        "Times Cited by Year": ["Year", "Times Cited"],
        "Cum. Times Cited by Year": ["Year", "Times Cited (Cum)"],
        "Avg. Times Cited by Year": ["Year", "Avg. Times Cited"],
    }
    plots = {'bar': plt.bar, "barh": plt.bar}
    #
    selected = widgets.Dropdown(
        options=list(data.keys()), value=list(data.keys())[0], description="Plot:", disable=False,
    )
    plot_type = widgets.Dropdown(
        options=["bar", "barh"], description="Plot type:", disable=False,
    )
    cmap = widgets.Dropdown(options=COLORMAPS, description="Colormap:", disable=False,)
    #
    output = widgets.Output()
    with output:
        display(
            widgets.interactive_output(
                compute, {"selected": selected, "plot_type": plot_type, "cmap": cmap}
            )
        )
    #
    left_box = widgets.VBox(
        [selected, plot_type, cmap], layout=Layout(height="350px", border="1px solid gray"),
    )
    right_box = widgets.VBox([output])
    return widgets.HBox([left_box, right_box])

In [170]:
def dashboard(x):
    m = [
        (menu_descriptive_stats, "Descriptive statistics"),
        (menu_analysis_by_year, "Analysis by year"),
        (menu_analysis_by_term, "Analysis by term"),
        (menu_analysis_by_term_per_year, "Analysis by term per year"),
        (menu_occurrence, "Occurrence analysis"),
        (menu_co_occurrence, "Co-occurrence analysis"),
        (menu_autocorrelation, "Autocorrelation analysis"),
        (menu_correlation, "Correlation analysis"),
        (menu_factor_analysis, "Factor analysis"),
    ]
    accordion = widgets.Accordion()
    widget_list = []
    for f, _ in m:
        widget_list.append(f(x))
    accordion.children = widget_list
    for index, t in enumerate(m):
        accordion.set_title(index, t[1])
    return accordion


dashboard(df)

Accordion(children=(Text(value='TO-DO'), HBox(children=(VBox(children=(Dropdown(description='Plot:', options=(…

In [96]:
widgets.Dropdown(
        options=["1", "2", "3"], value="1", description="Descrption:", disable=False,
    ).style

DescriptionStyle()

In [ ]:
def f(m, n, o):
    print('{}*{}*{}={}'.format(m, n, o, m*n*o))



a = widgets.IntSlider(description='a')
b = widgets.IntSlider(description='b')
c = widgets.IntSlider(description='c')

out = widgets.interactive_output(f, {'m': a, 'n': b, 'o': c})

widgets.HBox([widgets.VBox([a, b, c]), out])

In [ ]:
print(widgets.Dropdown.on_click.__doc__)

In [ ]:
jpl.summary_by_term(df, cmap="Blues")

In [ ]:
button = widgets.Button(description='Click here')
slider = widgets.IntSlider()
stacked = widgets.Stacked([button, slider])
stacked

In [ ]:
import ipywidgets as widgets

button = widgets.Button(description="Click here")
slider = widgets.IntSlider()
stacked = widgets.Stacked([button, slider])
stacked  # will show only the button

In [ ]:
tab_nest = widgets.Tab()
tab_nest.children = [accordion, accordion]
tab_nest.titles = ("An accordion", "Copy of the accordion")
tab_nest

In [ ]:
tab_contents = ["P0", "P1", "P2", "P3", "P4"]
children = [widgets.Text(description=name) for name in tab_contents]
tab = widgets.Tab()
tab.children = children
tab.titles = [str(i) for i in range(len(children))]
tab

In [ ]:
accordion = widgets.Accordion(
    children=[widgets.IntSlider(), widgets.Text()], titles=("Slider", "Text")
)
accordion

In [ ]:
# jpl.summary_by_year(df, cmap='Blues')

In [173]:
x = {
    "Abbreviated Source Title": "J9",
    "Abstract": "AB",
    "Access Type": "OA",
    "Affiliations": "C1",
    "Art. No.": "AR",
    "Author Keywords": "DE",
    "Author(s) Country": "AU_CO",
    "Author(s) ID": "RI",
    "Author(s) Institution": "AU_IN",
    "Authors with affiliations": "AU_C1",
    "Authors": "AU",
    "Cited by": "TC",
    "Cited references": "CR",
    "DI": "DOI",
    "Document type": "DT",
    "Editors": "BE",
    "EID": "UT",
    "Index Keywords": "ID",
    "ISBN": "BN",
    "ISSN": "SN",
    "Issue": "IS",
    "Keywords": "AU_ID",
    "Language of Original Document": "LA",
    "Page count": "PG",
    "Page end": "EP",
    "Page start": "BP",
    "Publisher": "PU",
    "PubMed ID": "PM",
    "Source title": "SO",
    "Source": "FN",
    "Subject": "SC",
    "Title": "TI",
    "Volume": "VL",
    "Year": "PY",
}

for a in x.keys():
    print('"', x[a],'": "', a,'"', sep="")

"J9": "Abbreviated Source Title"
"AB": "Abstract"
"OA": "Access Type"
"C1": "Affiliations"
"AR": "Art. No."
"DE": "Author Keywords"
"AU_CO": "Author(s) Country"
"RI": "Author(s) ID"
"AU_IN": "Author(s) Institution"
"AU_C1": "Authors with affiliations"
"AU": "Authors"
"TC": "Cited by"
"CR": "Cited references"
"DOI": "DI"
"DT": "Document type"
"BE": "Editors"
"UT": "EID"
"ID": "Index Keywords"
"BN": "ISBN"
"SN": "ISSN"
"IS": "Issue"
"AU_ID": "Keywords"
"LA": "Language of Original Document"
"PG": "Page count"
"EP": "Page end"
"BP": "Page start"
"PU": "Publisher"
"PM": "PubMed ID"
"SO": "Source title"
"FN": "Source"
"SC": "Subject"
"TI": "Title"
"VL": "Volume"
"PY": "Year"


In [174]:
z = ["AU", "DE", "ID", "RI", "C1", "DE_ID", "AU_CO", "AU_IN", "AU_C1"]

In [177]:
NORMALIZED_NAMES = {
    "J9": "Abbreviated Source Title",
    "AB": "Abstract",
    "OA": "Access Type",
    "C1": "Affiliations",
    "AR": "Art. No.",
    "DE": "Author Keywords",
    "AU_CO": "Author(s) Country",
    "RI": "Author(s) ID",
    "AU_IN": "Author(s) Institution",
    "AU_C1": "Authors with affiliations",
    "AU": "Authors",
    "TC": "Cited by",
    "CR": "Cited references",
    "DOI": "DI",
    "DT": "Document type",
    "BE": "Editors",
    "UT": "EID",
    "ID": "Index Keywords",
    "BN": "ISBN",
    "SN": "ISSN",
    "IS": "Issue",
    "DE_ID": "Keywords",
    "LA": "Language of Original Document",
    "PG": "Page count",
    "EP": "Page end",
    "BP": "Page start",
    "PU": "Publisher",
    "PM": "PubMed ID",
    "SO": "Source title",
    "FN": "Source",
    "SC": "Subject",
    "TI": "Title",
    "VL": "Volume",
    "PY": "Year",
}


In [180]:
for e in z:
    print('"', NORMALIZED_NAMES[e],'",', sep='')

"Authors",
"Author Keywords",
"Index Keywords",
"Author(s) ID",
"Affiliations",
"Keywords",
"Author(s) Country",
"Author(s) Institution",
"Authors with affiliations",


In [ ]:
NORMALIZED_NAMES = {
    "J9": "Abbreviated Source Title",
    "AB": "Abstract",
    "OA": "Access Type",
    "C1": "Affiliations",
    "AR": "Art. No.",
    "DE": "Author Keywords",
    "AU_CO": "Author(s) Country",
    "RI": "Author(s) ID",
    "AU_IN": "Author(s) Institution",
    "AU_C1": "Authors with affiliations",
    "AU": "Authors",
    "TC": "Cited by",
    "CR": "Cited references",
    "DOI": "DI",
    "DT": "Document type",
    "BE": "Editors",
    "UT": "EID",
    "ID": "Index Keywords",
    "BN": "ISBN",
    "SN": "ISSN",
    "IS": "Issue",
    "DE_ID": "Keywords",
    "LA": "Language of Original Document",
    "PG": "Page count",
    "EP": "Page end",
    "BP": "Page start",
    "PU": "Publisher",
    "PM": "PubMed ID",
    "SO": "Source title",
    "FN": "Source",
    "SC": "Subject",
    "TI": "Title",
    "VL": "Volume",
    "PT": "Year",
}